In [ ]:
import numpy as np
#tensorflow packages
# !pip install tensorflow
import tensorflow as tf
# !pip install tensorflow-hub
# !pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds



In [ ]:
import tensorflow_datasets as tfds

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)


In [63]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [64]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))


In [65]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [66]:
 train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [67]:
import tensorflow_hub as hub
import tensorflow as tf

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)


In [68]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [69]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [70]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_8 (Dense)             (None, 16)                336       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [72]:

# Now you can proceed with training
history = model.fit(
    train_data.shuffle(10000).batch(100),
    epochs=25,
    validation_data=validation_data.batch(100),
    verbose=1
)


Epoch 1/25
150/150 [==============================] - 7s 34ms/step - loss: 0.5735 - accuracy: 0.6806 - val_loss: 0.5236 - val_accuracy: 0.7023
Epoch 2/25
150/150 [==============================] - 4s 27ms/step - loss: 0.4651 - accuracy: 0.7663 - val_loss: 0.4344 - val_accuracy: 0.7845
Epoch 3/25
150/150 [==============================] - 4s 27ms/step - loss: 0.3792 - accuracy: 0.8237 - val_loss: 0.3756 - val_accuracy: 0.8264
Epoch 4/25
150/150 [==============================] - 4s 27ms/step - loss: 0.3117 - accuracy: 0.8644 - val_loss: 0.3403 - val_accuracy: 0.8418
Epoch 5/25
150/150 [==============================] - 4s 27ms/step - loss: 0.2675 - accuracy: 0.8877 - val_loss: 0.3201 - val_accuracy: 0.8498
Epoch 6/25
150/150 [==============================] - 4s 27ms/step - loss: 0.2284 - accuracy: 0.9093 - val_loss: 0.3085 - val_accuracy: 0.8566
Epoch 7/25
150/150 [==============================] - 6s 38ms/step - loss: 0.1988 - accuracy: 0.9233 - val_loss: 0.3009 - val_accuracy: 0.8683

In [73]:
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))


250/250 - 3s - loss: 0.6397 - accuracy: 0.8430 - 3s/epoch - 11ms/step
loss: 0.640
accuracy: 0.843
